In [ ]:
import os
import json

files = os.listdir('REVERIE')
with open("REVERIE/BBoxes.json", 'r') as f:
    bboxes = json.load(f)

image_info = {}
for file in files:
    if "BBoxes" in file or "aug" in file:
        continue    
    with open(os.path.join('REVERIE', file), 'r') as f:
        reverie_data = json.load(f)
        for path in reverie_data:
            scan = path['scan']
            pano = path["path"][-1]
            index = f"{scan}_{pano}"
            obj_id = path['objId']
            obj_info = bboxes[index][str(obj_id)]
            obj_info['scan'] = scan
            obj_info['pano'] = pano

            if path["id"] not in image_info:
                image_info[path["id"]] = obj_info
            elif "train_seen" not in file:
                raise Exception("Duplicate image id")
            
with open("image_info.json", 'w') as f:
    json.dump(image_info, f, indent=4, sort_keys=True)



In [26]:
import cv2
from tqdm import tqdm
with open("image_info.json", 'r') as f:
    image_info = json.load(f)

missing_obj = {}
for k, v in tqdm(image_info.items()):
    scan = v['scan']
    pano = v['pano']
    poses = v['visible_pos']
    bboxes = v['bbox2d']

    nums = len(poses)
    assert nums == len(bboxes)
    max_area = 0

    if nums == 0:
        missing_obj[k] = v
        continue
    
    for i in range(nums):
        pose = poses[i]
        bbox = bboxes[i]
        view_path = os.path.join("/mnt/data/haodong/envedit/views_img/", scan, pano, f"{pose}.jpg")
        assert os.path.exists(view_path)
        image = cv2.imread(view_path)
        x, y, w, h = bbox
        
        # calculate bbox area
        area = w * h
        if area > max_area:
            max_area = area
            max_idx = pose

        Height, Width = image.shape[:2]
        # Expand the bbox for 50 pixels
        expand = 30
        x1 = max(0, x-expand)
        y1 = max(0, y-expand)
        x2 = min(Width, x+w+expand)
        y2 = min(Height, y+h+expand)

        # crop the image
        crop_img = image[y1:y2, x1:x2]

        # save the image
        save_path = os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k, f"{pose}.jpg")
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        cv2.imwrite(save_path, crop_img)

    # rename the image with max area
    os.rename(os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k, f"{max_idx}.jpg"), os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k, f"{max_idx}_max.jpg"))

with open("missing_obj.json", 'w') as f:
    json.dump(missing_obj, f, indent=4, sort_keys=True)

100%|██████████| 5993/5993 [05:00<00:00, 19.94it/s]  


In [61]:
from copy import deepcopy

with open("missing_obj.json", 'r') as f:
    missing_obj = json.load(f)

filter_obj = {}
for k, v in tqdm(missing_obj.items()):
    bboxes = v['bbox2d']
    pano = v['pano']
    scan = v['scan']
    poses = v['visible_pos']
    labels = v['lable']
    name = v['name']

    nums = len(poses)
    qualified = 0

    filter_obj[k] = deepcopy(v)
    filter_obj[k]['visible_pos'] = []
    filter_obj[k]['bbox2d'] = []
    filter_obj[k]['label'] = []

    score_dict = {}
    for i in range(nums):
        bbox = bboxes[i]
        pose = poses[i]
        label = labels[i]

        label_name = label.split('(')[0]
        score = float(label.split('(')[1][:-1])

        if label_name != name:
            continue
        
        if score < 0.4 and k != "5494_385":
            continue

        if pose not in filter_obj[k]['visible_pos']:
            filter_obj[k]['visible_pos'].append(pose)
            filter_obj[k]['bbox2d'].append(bbox)
            filter_obj[k]['label'].append(score)
            score_dict[pose] = score
        else:
            if score > score_dict[pose]:
                score_dict[pose] = score
                idx = filter_obj[k]['visible_pos'].index(pose)
                filter_obj[k]['bbox2d'][idx] = bbox
                filter_obj[k]['label'][idx] = score

    if len(filter_obj[k]['visible_pos']) == 0:
        print(k)

with open("filter_obj.json", 'w') as f:
    json.dump(filter_obj, f, indent=4, sort_keys=True)
               


100%|██████████| 85/85 [00:00<00:00, 5899.85it/s]

In [70]:
with open("filter_obj.json", 'r') as f:
    filter_obj = json.load(f)

for k, v in tqdm(filter_obj.items()):
    scan = v['scan']
    pano = v['pano']
    poses = v['visible_pos']
    bboxes = v['bbox2d']
    scores = v['label']

    nums = len(poses)
    assert nums == len(bboxes)
    max_score = 0

    save_path = os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k)
    existing_files = len([x for x in os.listdir(save_path) if "max" in x])
    if existing_files == 1:
        continue

    for i in range(nums):
        pose = poses[i]
        bbox = bboxes[i]
        score = scores[i]
        view_path = os.path.join("/mnt/data/haodong/envedit/views_img/", scan, pano, f"{pose}.jpg")
        assert os.path.exists(view_path)
        image = cv2.imread(view_path)
        x1, y1, x2, y2 = bbox
        
        # calculate bbox area
        if score > max_score:
            max_score = score
            max_idx = pose

        Height, Width = image.shape[:2]
        # Expand the bbox for x pixels
        expand = 30
        x1 = int(max(0, x1-expand))
        y1 = int(max(0, y1-expand))
        x2 = int(min(Width, x2+expand))
        y2 = int(min(Height, y2+expand))

        # crop the image
        crop_img = image[y1:y2, x1:x2]

        # save the image
        save_path = os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k, f"{pose}.jpg")
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        cv2.imwrite(save_path, crop_img)

    # rename the image with max area
    os.rename(os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k, f"{max_idx}.jpg"), os.path.join("/mnt/data/haodong/REVERIE/obj_views", scan, pano, k, f"{max_idx}_max.jpg"))

100%|██████████| 85/85 [00:04<00:00, 19.92it/s]


In [66]:
with open("filter_obj.json", 'r') as f:
    filter_obj = json.load(f)

for k, v in tqdm(filter_obj.items()):
    scan = v['scan']
    pano = v['pano']
    if len(v['visible_pos']) >= 10:
        print(f"/mnt/data/haodong/REVERIE/obj_views/{scan}/{pano}/{k}", v["name"])

100%|██████████| 85/85 [00:00<00:00, 61616.98it/s]

/mnt/data/haodong/REVERIE/obj_views/b8cTxDM8gDG/031a6316a244425c8c9a7a43ab6de08d/1205_472 shower wall
/mnt/data/haodong/REVERIE/obj_views/cV4RVeZvu5T/3393cfe75d8e4593bf4b05d00b7eb2c2/1276_316 faucet
/mnt/data/haodong/REVERIE/obj_views/V2XKFyX4ASd/ff5537fd1daf4f13abf85e6dda34aba5/1331_177 towel
/mnt/data/haodong/REVERIE/obj_views/mJXqzFtmKg4/929158cc8c424c099a601a36e0fc2270/1513_273 picture
/mnt/data/haodong/REVERIE/obj_views/JmbYfDe2QKZ/e4753c26daf64c8492bb3fad5d959fb3/1524_161 chair
/mnt/data/haodong/REVERIE/obj_views/D7N2EKCX4Sj/46e170296a7f4b88b50ebeb7e2a27894/1527_213 picture
/mnt/data/haodong/REVERIE/obj_views/7y3sRwLe3Va/1838a8fb8e9f48cebcd079d5373cba02/1714_538 thermostat
/mnt/data/haodong/REVERIE/obj_views/V2XKFyX4ASd/0c17fd1d27164de4ad2274e6e0d64a89/209_298 window
/mnt/data/haodong/REVERIE/obj_views/7y3sRwLe3Va/3ef76f35c6ae4ee6a69122e2413900e5/2272_300 light
/mnt/data/haodong/REVERIE/obj_views/XcA2TqTSSAj/7d4680fa7e9d4770971d94b194ed6a87/2426_153 window
/mnt/data/haodong/REVER